# Key term extraction

This tutorial shows how to extract key terms from document and (sub-)collections with TF-IDF and the log-likelihood statistic and a reference corpus.

1. TF-IDF
2. Log-likelihood ratio test
3. Aggregations and visualization

Like in the previous tutorial we read the CSV data file containing the job posts and preprocess the corpus object with a sequence of `quanteda` functions.

This time, we also apply **lemmatization** to the corpus. Lemmatization reduces (potentially) inflected word forms to its lexical lemma to unify similar semantic forms to the same text representation. For instance, 'presidents' becomes '' and 'singing' becomes 'sing'.

Finally, we create a Document-Term-Matrix.

In [ ]:
options(stringsAsFactors = FALSE, digits = 3)
require(quanteda)
require(magrittr)
require(dplyr)
require(data.table)

In [ ]:
# read csv into a data.frame
textdata <- read.csv("data/data job posts.csv", header = TRUE, sep = ",", encoding = "UTF-8",quote = "\"")
textdata <- as.data.table(textdata)

english_stopwords <- readLines("data/stopwords_en.txt", encoding = "UTF-8")

textdata %<>% filter(!duplicated(jobpost))
textdata %<>% mutate(d_id = 1:nrow(textdata))

#Build a dictionary of lemmas
lemmaData <- read.csv2("data/baseform_en.tsv", sep="\t", header=FALSE, encoding = "UTF-8", stringsAsFactors = F)

data_corpus <- corpus(textdata$jobpost, docnames = textdata$d_id)

# Create a DTM (may take a while)
data_dfm_entries <- data_corpus %>% tokens() %>%
  tokens(remove_punct = TRUE, remove_numbers = TRUE, remove_symbols = TRUE) %>% tokens_tolower() %>% 
  tokens_replace(., lemmaData$V1, lemmaData$V2) %>%
  tokens_ngrams(1) %>% tokens_remove(pattern = stopwords()) %>% dfm() 


data_dfm_entries_sub <- data_dfm_entries %>%
  dfm_select(pattern = "[a-z]", valuetype = "regex", selection = 'keep')

colnames(data_dfm_entries_sub) <- colnames(data_dfm_entries_sub) %>% stringi::stri_replace_all_regex("[^_a-z]", "") 

DTM <- dfm_compress(data_dfm_entries_sub, "features")
# Show some information
DTM

## TF-IDF

A widely used method to weight terms according to their semantic contribution to a document is the **term frequency--inverse document frequency** measure (TF-IDF). The idea is, the more a term occurs in a document, the more contributing it is. At the same time, in the more documents a term occurs, the less informative it is for a single document. The product of both measures is the resulting weight.

Let us compute TF-IDF weights for all terms.

In [ ]:
# Compute IDF: log(N / n_i)
number_of_docs <- nrow(DTM)
term_in_docs <- colSums(DTM > 0)
idf <- log2(number_of_docs / term_in_docs)

# Compute TF
jobpost <- which(textdata$Company == "Samsung Electronics Representative Office in Armenia")[1]
tf <- as.vector(DTM[jobpost, ])

# Compute TF-IDF
tf_idf <- tf * idf
names(tf_idf) <- colnames(DTM)

The last operation is to append the column names again to the resulting term weight vector. If we now sort the tf-idf weights decreasingly, we get the most important terms for the job post, according to this weight.

In [ ]:
sort(tf_idf, decreasing = T)[1:20]

If we would have just relied upon term frequency, we would have obtained a list of stop words as most important terms. By re-weighting with inverse document frequency, we can see a hmore senseful word list. By the way, the quanteda-package provides a convenient function for computing tf-idf weights of a given DTM: `weightTfIdf(DTM)`.

## Log likelihood

We now use a more sophisticated method with a comparison corpus and the log likelihood statistic.


In [ ]:
targetDTM <- DTM

termCountsTarget <- as.vector(targetDTM[jobpost, ])
names(termCountsTarget) <- colnames(targetDTM)
# Just keep counts greater than zero
termCountsTarget <- termCountsTarget[termCountsTarget > 0]

In *termCountsTarget* we have the tf for the document again.

As a comparison corpus, we select a corpus from the Leipzig Corpora Collection (http://corpora.uni-leipzig.de): 30.000 randomly selected sentences from the Wikipedia of 2010. 
**CAUTION:** The preprocessing of the comparison corpus must be identical to the preprocessing Of the target corpus to achieve meaningful results!

In [ ]:
lines <- data.frame(text=readLines("resources/eng_wikipedia_2010_30K-sentences.txt", encoding = "UTF-8"))

lines %<>% mutate(d_id = 1:nrow(lines))

#Build a dictionary of lemmas
data_corpus_reference <- corpus(lines$text, docnames = lines$d_id)

# Create a DTM (may take a while)
data_dfm_entries_reference <- data_corpus_reference %>% tokens() %>%
  tokens(remove_punct = TRUE, remove_numbers = TRUE, remove_symbols = TRUE) %>% tokens_tolower() %>% 
  tokens_replace(., lemmaData$V1, lemmaData$V2) %>%
  tokens_ngrams(1) %>% tokens_remove(pattern = stopwords()) %>% dfm() 


data_dfm_entries_sub_reference <- data_dfm_entries_reference %>%
  dfm_select(pattern = "[a-z]", valuetype = "regex", selection = 'keep')

colnames(data_dfm_entries_sub_reference) <- colnames(data_dfm_entries_sub_reference) %>% stringi::stri_replace_all_regex("[^_a-z]", "") 

DTM_reference <- dfm_compress(data_dfm_entries_sub_reference, "features")
# Show some information
DTM_reference

From the comparison corpus, we also create a count of all terms. 

In [ ]:
comparisonDTM <- DTM_reference
termCountsComparison <- colSums(comparisonDTM)

In *termCountsComparison* we now have the frequencies of all (target) terms in the comparison corpus.

Let us now calculate the log-likelihood ratio test by comparing frequencies of a term in both corpora, taking the size of both corpora into account. First for a single term:

In [ ]:
# Loglikelihood for a single term
term <- "representative"

# Determine variables
a <- termCountsTarget[term]
b <- termCountsComparison[term]
c <- sum(termCountsTarget)
d <- sum(termCountsComparison)

# Compute log likelihood test
Expected1 = c * (a+b) / (c+d)
Expected2 = d * (a+b) / (c+d)
t1 <- a * log((a/Expected1))
t2 <- b * log((b/Expected2))
logLikelihood <- 2 * (t1 + t2)

print(logLikelihood)

The LL value indicates whether the term occurs significantly more frequently / less frequently in the target counts than we would expect from the observation in the comparative counts. Specific significance thresholds are defined for the LL values:

- 95th percentile; 5% level; p < 0.05; critical value = 3.84
- 99th percentile; 1% level; p < 0.01; critical value = 6.63
- 99.9th percentile; 0.1% level; p < 0.001; critical value = 10.83
- 99.99th percentile; 0.01% level; p < 0.0001; critical value = 15.13 

With R it is easy to calculate the LL-value for all terms at once. This is possible because many computing operations in R can be applied not only to individual values, but to entire vectors and matrices. For example, `a / 2` results in a single value *a divided by 2* if `a` is a single number. If `a` is a vector, the result is also a vector, in which all values are divided by 2.

ATTENTION: A comparison of term occurrences between two documents/corpora is actually only useful if the term occurs in both units. Since, however, we also want to include terms which are not contained in the comparative corpus (the `termCountsComparison` vector contains 0 values for these terms), we simply add 1 to all counts during the test. This is necessary to avoid `NaN` values which otherwise would result from the log-function on 0-values during the LL test. Alternatively, the test could be performed only on terms that actually occur in both corpora.

First, let's have a look into the set of terms only occurring in the target document, but not in the comparison corpus.


In [ ]:
# use set operation to get terms only occurring in target document
uniqueTerms <- setdiff(names(termCountsTarget), names(termCountsComparison))
# Have a look into a random selection of terms unique in the target corpus
sample(uniqueTerms, 2)

Now we calculate the statistics the same way as above, but with vectors. But, since there might be terms in the targetCounts which we did not observe in the comparison corpus, we need to make both vocabularies matching. For this, we append unique terms from the target as zero counts to the comparison frequency vector. Moreover, we use a little trick to check for zero counts of frequency values in a or b when computing t1 or t2. If a count is zero the log function would produce an NaN value, which we want to avoid. In this case the `a == 0` resp. `b == 0` expression add 1 to the expression which yields a 0 value after applying the log function.

In [ ]:
# Create vector of zeros to append to comparison counts
zeroCounts <- rep(0, length(uniqueTerms))
names(zeroCounts) <- uniqueTerms
termCountsComparison <- c(termCountsComparison, zeroCounts)

# Get list of terms to compare from intersection of target and comparison vocabulary
termsToCompare <- intersect(names(termCountsTarget), names(termCountsComparison))

# Calculate statistics (same as above, but now with vectors!)
a <- termCountsTarget[termsToCompare]
b <- termCountsComparison[termsToCompare]
c <- sum(termCountsTarget)
d <- sum(termCountsComparison)
Expected1 = c * (a+b) / (c+d)
Expected2 = d * (a+b) / (c+d)
t1 <- a * log((a/Expected1) + (a == 0))
t2 <- b * log((b/Expected2) + (b == 0))
logLikelihood <- 2 * (t1 + t2)

# Compare relative frequencies to indicate over/underuse
relA <- a / c
relB <- b / d
# underused terms are multiplied by -1
logLikelihood[relA < relB] <- logLikelihood[relA < relB] * -1

Let's take a look at the results: The 50 more frequently used / less frequently used terms, and then the more frequently used terms compared to their frequency. We also see terms that have comparatively low frequencies are identified by the LL test as statistically significant compared to the reference corpus.

In [ ]:
# top terms (overuse in targetCorpus compared to comparisonCorpus)
sort(logLikelihood, decreasing=TRUE)[1:25]
# bottom terms (underuse in targetCorpus compared to comparisonCorpus)
sort(logLikelihood, decreasing=FALSE)[1:25]

llTop100 <- sort(logLikelihood, decreasing=TRUE)[1:100]
frqTop100 <- termCountsTarget[names(llTop100)]
frqLLcomparison <- data.frame(llTop100, frqTop100)
frqLLcomparison

In [ ]:
# Number of signficantly overused terms (p < 0.01)
sum(logLikelihood > 6.63)

## Aggregation and visualization

Finally, visualize the result of the 50 most significant terms as Wordcloud. This can be realized simply by function of the package wordcloud. Additionally to the words and their weights (here we use likelihood values), we override default scaling and color parameters. Feel free to try different parameters to modify the wordcloud rendering.

In [ ]:
require(wordcloud)
top50 <- sort(logLikelihood, decreasing = TRUE)[1:50]
wordcloud(names(top50), top50, max.words = 50, scale = c(3, .9), colors = brewer.pal(8, "Dark2"), random.order = F)

Key term extraction cannot be done for single documents, but for entire (sub-)corpora. Depending on the comparison corpora, the results may vary. Instead of comparing a single document to a Wikipedia corpus, we now compare collections of job posts of a single company, to job posts of all other companies.

For this, we iterate over all different company names using a for-loop. Within the loop, we utilize a logical vector (Boolean TRUE/FALSE values), to split the DTM into two sub matrices: rows of the currently selected company and rows of all other companies. From these matrices our counts of target and comparison frequencies are created. The statistical computation of the log-likelihood measure from above, we outsourced into the function `calculateLogLikelihood` which we load with the `source` command at the beginning of the block. The function just takes both frequency vectors as input parameters and outputs a LL-value for each term of the target vector.

Results of the LL key term extraction are visualized again as a wordcloud. Instead of plotting the wordcloud into RStudio, this time we write the visualization as a PDF-file to disk into the `wordclouds` folder. After the for-loop is completed, the folder should contain numerous wordcloud PDFs, one for each company.

In [ ]:
source("calculateLogLikelihood.R")

companies <- unique(textdata$Company)
for (company in companies) {
  
  cat("Extracting terms for company", company, "\n")
  
  selector_logical_idx <- textdata$Company == company
  
  companyDTM <- targetDTM[selector_logical_idx, ]
  termCountsTarget <- colSums(companyDTM)
  
  otherDTM <- targetDTM[!selector_logical_idx, ]
  termCountsComparison <- colSums(otherDTM)
  
  loglik_terms <- calculateLogLikelihood(termCountsTarget, termCountsComparison)
  
  top50 <- sort(loglik_terms, decreasing = TRUE)[1:50]
  
  fileName <- paste0("wordclouds-", company, ".pdf")
  pdf(fileName, width = 9, height = 7)
  wordcloud(names(top50), top50, max.words = 50, scale = c(3, .9), colors = brewer.pal(8, "Dark2"), random.order = F)
  dev.off()
  
}